In [31]:
import pandas as pd
import numpy as np
import seaborn as sns

# Read data

In [32]:
jobs = pd.read_csv("job_training_data.csv")
jobs

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function.,fraudulent
0,13454,Field Operations Specialist,"US, SD, Yankton",NaN,NaN,"Working at FBNAt FBN, we put farmers first. Fo...",Farmers Business Network (“FBN”) is an advance...,"Things we look forHonesty, integrity and a pas...",Competitive compensation based on experienceHe...,0,1,0,Full-time,Director,Bachelor's Degree,Farming,Customer Service,0
1,11411,CM2 5AS Warehousing apprenticeship available u...,"GB, , Chelmsford",NaN,NaN,Established on the principles that full time e...,This is fantastic opportunity for someone want...,Government funding is only available for 16-18...,Future Prospects,0,1,1,NaN,Not Applicable,High School or equivalent,NaN,NaN,0
2,15153,Lead QA Tester,"GB, LND, London",Blippbuilder,35000-40000,"As augmented reality jobs go, one at Blippar i...",We are looking for an experienced Lead QA test...,This role requires somebody who has both a str...,"Our bright, open, centrally located offices ar...",0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Quality Assurance,0
3,2814,Basingstoke Fragrance/Beauty Staff Needed,"GB, HAM, Basingstoke",NaN,NaN,Established on the principles that full time e...,We are currently recruiting for an exciting Pr...,NaN,NaN,0,1,0,NaN,NaN,NaN,Cosmetics,NaN,0
4,13996,Swim Instructor,"US, NJ, Hawthorne",NaN,NaN,NaN,Looking for a part time swimming instructor to...,Must be Lifeguard and CPR certified,Pay is negotiable,0,0,1,Part-time,NaN,Unspecified,NaN,Other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5357,17632,Property Preservation Field Crews,"US, MD, Frederick",NaN,NaN,NaN,We are looking for a contractors/individual wh...,**NOTE: You must have the necessary tools for...,As per job,0,0,0,Contract,NaN,NaN,Real Estate,NaN,1
5358,17751,Administrative Assistant,"US, MD, Baltimore",NaN,NaN,NaN,Administrative AssistantEssential Job Responsi...,NaN,NaN,0,0,0,Full-time,Entry level,Unspecified,Hospital & Health Care,Administrative,1
5359,17561,Job for freelancer working in Saudi Arabia,"SA, 01, Murabba Riyadh",IT Administrator / Developer,NaN,NaN,Hundreds of Freelance Jobs available every mon...,Freelancer working in Saudi Arabia.,NaN,0,0,1,Full-time,Entry level,Master's Degree,Computer Networking,Information Technology,1
5360,2397,Senior QA Engineer,"IN, , Hyderabad",Information Technology,NaN,Aptitude Staffing Solutions has redesigned the...,Responsibilities:* Acquire and maintain a comp...,Required Skills and Experience:Minimum of five...,What Is OfferedOur client's core values drive ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Marketing and Advertising,Engineering,1


# split by is fraud or not

In [3]:
fraud = jobs[jobs["fraudulent"] == 1]
fraud.to_csv("fraudulent.csv",index = False)
not_fraud = jobs[jobs["fraudulent"] == 0]
not_fraud.to_csv("not_fraudulent.csv",index = False)

In [34]:
jobs.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function.',
       'fraudulent'],
      dtype='object')

# length counter

In [46]:
def length_counter(df,columns = ["company_profile","description","requirements","benefits"]):
    length = []
    for column in columns:
        df[(str(column) + "length")] = df[column].apply(lambda x: len(x) if x == x else 0)
    return df

In [47]:
jobs = length_counter(jobs)

In [48]:
jobs.

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,employment_type,required_experience,required_education,industry,function.,fraudulent,company_profilelength,descriptionlength,requirementslength,benefitslength
0,13454,Field Operations Specialist,"US, SD, Yankton",NaN,NaN,"Working at FBNAt FBN, we put farmers first. Fo...",Farmers Business Network (“FBN”) is an advance...,"Things we look forHonesty, integrity and a pas...",Competitive compensation based on experienceHe...,0,...,Full-time,Director,Bachelor's Degree,Farming,Customer Service,0,2559,871,1383,100
1,11411,CM2 5AS Warehousing apprenticeship available u...,"GB, , Chelmsford",NaN,NaN,Established on the principles that full time e...,This is fantastic opportunity for someone want...,Government funding is only available for 16-18...,Future Prospects,0,...,NaN,Not Applicable,High School or equivalent,NaN,NaN,0,829,391,88,16
2,15153,Lead QA Tester,"GB, LND, London",Blippbuilder,35000-40000,"As augmented reality jobs go, one at Blippar i...",We are looking for an experienced Lead QA test...,This role requires somebody who has both a str...,"Our bright, open, centrally located offices ar...",0,...,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Quality Assurance,0,1251,724,805,2047
3,2814,Basingstoke Fragrance/Beauty Staff Needed,"GB, HAM, Basingstoke",NaN,NaN,Established on the principles that full time e...,We are currently recruiting for an exciting Pr...,NaN,NaN,0,...,NaN,NaN,NaN,Cosmetics,NaN,0,829,838,0,0
4,13996,Swim Instructor,"US, NJ, Hawthorne",NaN,NaN,NaN,Looking for a part time swimming instructor to...,Must be Lifeguard and CPR certified,Pay is negotiable,0,...,Part-time,NaN,Unspecified,NaN,Other,0,0,92,35,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5357,17632,Property Preservation Field Crews,"US, MD, Frederick",NaN,NaN,NaN,We are looking for a contractors/individual wh...,**NOTE: You must have the necessary tools for...,As per job,0,...,Contract,NaN,NaN,Real Estate,NaN,1,0,421,651,10
5358,17751,Administrative Assistant,"US, MD, Baltimore",NaN,NaN,NaN,Administrative AssistantEssential Job Responsi...,NaN,NaN,0,...,Full-time,Entry level,Unspecified,Hospital & Health Care,Administrative,1,0,874,0,0
5359,17561,Job for freelancer working in Saudi Arabia,"SA, 01, Murabba Riyadh",IT Administrator / Developer,NaN,NaN,Hundreds of Freelance Jobs available every mon...,Freelancer working in Saudi Arabia.,NaN,0,...,Full-time,Entry level,Master's Degree,Computer Networking,Information Technology,1,0,336,35,0
5360,2397,Senior QA Engineer,"IN, , Hyderabad",Information Technology,NaN,Aptitude Staffing Solutions has redesigned the...,Responsibilities:* Acquire and maintain a comp...,Required Skills and Experience:Minimum of five...,What Is OfferedOur client's core values drive ...,0,...,Full-time,Mid-Senior level,Bachelor's Degree,Marketing and Advertising,Engineering,1,1099,930,418,2198
